<a href="https://colab.research.google.com/github/Ayush0345/Causal-Inference-Experiments/blob/main/DID_Est.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import combinations
import plotnine as p
import matplotlib.pyplot as plt

In [2]:
exp = pd.read_csv('/content/experimental.csv')
nexp = pd.read_csv('/content/nonexperimental.csv')

(a) Descriptive Statistics - Experimental Data

In [ ]:
exp75 = exp[exp['year']==75]

summary_stats = exp75.groupby('ever_treated').agg({
    'age': ['mean', 'median', 'std', 'min', 'max'],
    'educ': ['mean', 'median', 'std', 'min', 'max'],
    'marr': ['mean'],
    'nodegree': ['mean'],
    'black': ['mean'],
    'hisp': ['mean'],
    're74': ['mean', 'median', 'std', 'min', 'max'],
    'u74': ['mean']
})

# Rename the columns for better interpretation
summary_stats.columns = ['age_mean', 'age_median', 'age_std', 'age_min', 'age_max',
                         'educ_mean', 'educ_median', 'educ_std', 'educ_min', 'educ_max',
                         'marr_mean', 'nodegree_mean', 'black_mean', 'hisp_mean',
                         're74_mean', 're74_median', 're74_std', 're74_min', 're74_max',
                         'u74_mean']

print(summary_stats)

(b) Effect of treatment on real earning in 1978

In [4]:
# (i) Compute simple difference in means in year 1978
exp78 = exp[exp['year']==78]

treat = exp78[exp['ever_treated']==1]
control = exp78[exp['ever_treated']==0]

re_treat = treat.re
re_control = control.re

# Simple Difference in Means
sdm = re_treat.mean() - re_control.mean()
print(sdm)

1794.3423818501024


<ipython-input-4-6d848cc32fb8>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<ipython-input-4-6d848cc32fb8>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [5]:
# (ii) OLS using only the year 1978

re78 = exp78.re
treatment = exp78.ever_treated

model = sm.OLS(re78, treatment, data=exp).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                     re   R-squared (uncentered):                   0.233
Model:                            OLS   Adj. R-squared (uncentered):              0.231
Method:                 Least Squares   F-statistic:                              134.8
Date:                Thu, 25 Apr 2024   Prob (F-statistic):                    2.17e-27
Time:                        12:59:47   Log-Likelihood:                         -4597.9
No. Observations:                 445   AIC:                                      9198.
Df Residuals:                     444   BIC:                                      9202.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:130: ValueWarning: unknown kwargs ['data']


In [6]:
# (iii) Difference-in-Differences estimator using years 78 for post-period and 75 as the pre-period

re75 = exp75.re
re_treat75 = exp75[exp['ever_treated']==1]
re_control75 = exp75[exp['ever_treated']==0]

re_treat78 = exp78[exp['ever_treated']==1]
re_control78 = exp78[exp['ever_treated']==0]

avg_re75_treat = re_treat75.re75.mean()
avg_re75_control = re_control75.re75.mean()
avg_re78_treat = re_treat78.re.mean()
avg_re78_control = re_control78.re.mean()

did_est = (avg_re78_treat - avg_re75_treat) - (avg_re78_control - avg_re75_control)
print(did_est)

1529.1960833214434


<ipython-input-6-7c30fb115a17>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<ipython-input-6-7c30fb115a17>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<ipython-input-6-7c30fb115a17>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<ipython-input-6-7c30fb115a17>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(c) Use non-experimental dataset (nonexperimental.csv). Repeat (b)i, (b)ii, and (b)iii.

In [7]:
nexp78 = nexp[nexp['year']==78]
nexp75 = nexp[nexp['year']==75]

nre75 = nexp75.re
nre78 = nexp78.re

nre_treat75 = nexp75[nexp['ever_treated']==1]
nre_control75 = nexp75[nexp['ever_treated']==0]

nre_treat78 = nexp78[nexp['ever_treated']==1]
nre_control78 = exp78[nexp['ever_treated']==0]

avg_nre75_treat = nre_treat75.re.mean()
avg_nre75_control = nre_control75.re.mean()
avg_nre78_treat = nre_treat78.re.mean()
avg_nre78_control = nre_control78.re.mean()

did_est_nexp = (avg_nre78_treat - avg_nre75_treat) - (avg_nre78_control - avg_nre75_control)
print(did_est_nexp)

13913.090591182932


<ipython-input-7-d83ac4d936e8>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<ipython-input-7-d83ac4d936e8>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<ipython-input-7-d83ac4d936e8>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<ipython-input-7-d83ac4d936e8>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(d) Descriptive Statistics - Non-Experimental Data

In [8]:
nexp75 = nexp[nexp['year']==75]

nxp_summary_stats = nexp75.groupby('ever_treated').agg({
    'age': ['mean', 'median', 'std', 'min', 'max'],
    'educ': ['mean', 'median', 'std', 'min', 'max'],
    'marr': ['mean'],
    'nodegree': ['mean'],
    'black': ['mean'],
    'hisp': ['mean'],
    're74': ['mean', 'median', 'std', 'min', 'max'],
    'u74': ['mean']
})

# Rename the columns for better interpretation
nxp_summary_stats.columns = ['age_mean', 'age_median', 'age_std', 'age_min', 'age_max',
                         'educ_mean', 'educ_median', 'educ_std', 'educ_min', 'educ_max',
                         'marr_mean', 'nodegree_mean', 'black_mean', 'hisp_mean',
                         're74_mean', 're74_median', 're74_std', 're74_min', 're74_max',
                         'u74_mean']

print(nxp_summary_stats)

               age_mean  age_median    age_std  age_min  age_max  educ_mean  \
ever_treated                                                                  
0             33.225238        31.0  11.045216       16       55  12.027514   
1             25.816216        25.0   7.155019       17       48  10.345946   

              educ_median  educ_std  educ_min  educ_max  marr_mean  \
ever_treated                                                         
0                    12.0  2.870846         0        18   0.711731   
1                    11.0  2.010650         4        16   0.189189   

              nodegree_mean  black_mean  hisp_mean     re74_mean  \
ever_treated                                                       
0                  0.295835    0.073537   0.072036  14016.800360   
1                  0.708108    0.843243   0.059459   2095.573693   

               re74_median     re74_std  re74_min      re74_max  u74_mean  
ever_treated                                         

In [9]:
sm = summary_stats.mean()
nsm = nxp_summary_stats.mean()
change = sm-nsm
print(change)

age_mean           -4.085696
age_median         -3.500000
age_std            -1.993736
age_min             0.500000
age_max             0.000000
educ_mean          -0.969526
educ_median        -1.000000
educ_std           -0.628260
educ_min            1.500000
educ_max           -2.000000
marr_mean          -0.278942
nodegree_mean       0.269390
black_mean          0.376693
hisp_mean           0.017828
re74_mean       -5954.886854
re74_median     -7561.790039
re74_std        -1940.945162
re74_min            0.000000
re74_max         6854.179688
u74_mean            0.315189
dtype: float64


The results from the code in cell above will compute the difference between descriptive stats in experimental and non-experimental datasets (in terms of mean of covariates).


---



(e) DID estimator in the experimental dataset controlling for the vector of covariates described in cell 3.

In [10]:
data = pd.merge(exp75, exp78, on='id', suffixes=('75','78'))

# Creating a vector of covariates to control for in the regression model
covariates = pd.DataFrame({
    'age75': data['age75'],
    'age78' : data['age78'],
    'agesq75'  : data['agesq75'],
    'agesq78' : data['agesq78'],
    'agecube75': data['agecube75'],
    'agecube78': data['agecube78'],
    'educ75': data['educ75'],
    'educ78': data['educ78'],
    'educsq75': data['educsq75'],
    'educsq78': data['educsq78'],
    'marr75': data['marr75'],
    'marr78': data['marr78'],
    'black75': data['black75'],
    'black78': data['black78'],
    'hisp75': data['hisp75'],
    'hisp78': data['hisp78'],
    'nodegree75': data['nodegree75'],
    'nodegree78': data['nodegree78'],
    're7475': data['re7475'],
    're7478': data['re7478'],
    'u7475': data['u7475'],
    'u7478': data['u7478']
})

treatment = 'ever_treated78'
X = data[treatment]

diff = data['re78'] - data['re75']

model2 = smf.ols('diff ~ X + covariates', data=data).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                   diff   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     3.945
Date:                Thu, 25 Apr 2024   Prob (F-statistic):           8.66e-06
Time:                        13:00:00   Log-Likelihood:                -4553.8
No. Observations:                 445   AIC:                             9134.
Df Residuals:                     432   BIC:                             9187.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       7289.5122   1.48e+04      0.

(f) Using the DRDID command to estimate a doubly-robust difference-in-differences with covariates (Shifting to writing code in R for greater efficiency for DRDID package specifically)

In [ ]:
# install.packages("remotes")
remotes::install_github("pedrohcgs/DRDID")

In [22]:
library(DRDID)

data <- read.csv('/content/experimental.csv')

covariates_exp <- c(
    age=data$age,
    agesq=data$agesq,
    agecube=data$agecube,
    educ=data$educ,
    educsq=data$educsq,
    marr=data$marr,
    nodegree=data$nodegree,
    black=data$black,
    hisp=data$hisp,
    re74=data$re74,
    u74=data$u74)

# DRDID estimation

model3 <- drdid(yname = "re", tname = "year", idname = "id", dname = "ever_treated",
                xformla= ~ age+agesq+agecube+educ+educsq+marr+nodegree+black+hisp+re74+u74,
            data = data, panel = TRUE)

summary(model3)


 Call:
drdid(yname = "re", tname = "year", idname = "id", dname = "ever_treated", 
    xformla = ~age + agesq + agecube + educ + educsq + marr + 
        nodegree + black + hisp + re74 + u74, data = data, panel = TRUE)
------------------------------------------------------------------
 Further improved locally efficient DR DID estimator for the ATT:
 
   ATT     Std. Error  t value    Pr(>|t|)  [95% Conf. Interval] 
1549.1084   705.6317    2.1953     0.0281    166.0703  2932.1464 
------------------------------------------------------------------
 Estimator based on panel data.
 Outcome regression est. method: weighted least squares.
 Propensity score est. method: inverse prob. tilting.
 Analytical standard error.
------------------------------------------------------------------
 See Sant'Anna and Zhao (2020) for details.

Using the DID estimation in part (e), we find out that the average treatment effect on the treated (ATT) is ~ 1401. In part (f), we get the doubly-robust DID estimator measuring ATT to be ~ 1549.

Comparing to part (b), ATT from DRDID is sufficiently close to the DID estimator, i.e., 1549 ~ 1529.

However, in part (c) the DID estimator in the non-experimental data is significantly different from the ATT calculated using DRDID, i.e., 13913 != 1549. (!= ~ not equal)



---

